In [1]:
import pickle
import pandas as pd
import os
import sys
HOME = os.path.abspath('..')
sys.path.append(HOME)
os.chdir(HOME)
import torchvision
import dlib
from PIL import Image
from imutils import face_utils
from src.data.dataprep_mediapipe import *
from src.data.imgutils import *
from src.features.features_extractors_mediapipe import *
import pandas as pd
import seaborn as sns
from vidstab import VidStab
import matplotlib.pyplot as plt

In [56]:
v = cv2.VideoCapture('data/raw/3_face_smile.MOV')
a=[]
while True:
    ret, frame = v.read()
    if ret == False:
        break
    a.append(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
video = np.array(a)

In [3]:
v = pd.read_pickle('data/processed/29_face_smile.pkl')
video = [i['frame'] for i in v]

In [58]:


stabilizer = VidStab()
stabilizer.stabilize(input_path='data/raw/3_face_smile.MOV', output_path='data/raw/3_face_smile_stable.MOV')

In [59]:

v_s = cv2.VideoCapture('data/raw/3_face_smile_stable.MOV')
a_s=[]
while True:
    ret, frame = v_s.read()
    if ret == False:
        break
    a_s.append(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
video = np.array(a_s)


In [60]:
import cv2 as cv
import numpy as np


# The video feed is read in as
# a VideoCapture object
#cap = cv.VideoCapture('../data/dummy_data/samir/face_smile_sequence.mp4')

# ret = a boolean return value from
# getting the frame, first_frame = the
# first frame in the entire video sequence
first_frame = video[0]

#first_frame = cv2.resize(first_frame,(360,480))

# Converts frame to grayscale because we
# only need the luminance channel for
# detecting edges - less computationally
# expensive
prev_gray = cv.cvtColor(first_frame, cv.COLOR_RGB2GRAY)

#prev_gray = clahe.apply(prev_gray)

# Creates an image filled with zero
# intensities with the same dimensions
# as the frame
mask = np.zeros_like(first_frame)

# Sets image saturation to maximum
mask[..., 1] = 255

for frame in video[1:]:
    
    # ret = a boolean return value from getting
    # the frame, frame = the current frame being
    # projected in the video

    # Opens a new window and displays the input
    #frame = cv2.resize(frame,(360,480))

    cv.imshow("input", frame)
    
    # Converts each frame to grayscale - we previously
    # only converted the first frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    #gray = clahe.apply(gray)
    
    # Calculates dense optical flow by Farneback method
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray,
                                    None,
                                    0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Computes the magnitude and angle of the 2D vectors
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    
    # Sets image hue according to the optical flow
    # direction
    mask[..., 0] = angle * 180 / np.pi / 2
    
    # Sets image value according to the optical flow
    # magnitude (normalized)
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    
    # Converts HSV to RGB (BGR) color representation
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
    
    # Opens a new window and displays the output frame
    cv.imshow("dense optical flow", rgb)
    
    # Updates previous frame
    prev_gray = gray
    
    # Frames are read by intervals of 1 millisecond. The
    # programs breaks out of the while loop when the
    # user presses the 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break


KeyboardInterrupt: 

In [13]:
ret, first_frame = cap.read()

first_frame.shape

(720, 1280, 3)

In [14]:
720/2,1280/2

(360.0, 640.0)

In [6]:
cv.OPTFLOW_FARNEBACK_GAUSSIAN

256

In [3]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

# params for corner detection
feature_params = dict( maxCorners = 100,
					qualityLevel = 0.3,
					minDistance = 7,
					blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize = (15, 15),
				maxLevel = 2,
				criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
							10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame,
						cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None,
							**feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
	
	ret, frame = cap.read()
	frame_gray = cv2.cvtColor(frame,
							cv2.COLOR_BGR2GRAY)

	# calculate optical flow
	p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray,
										frame_gray,
										p0, None,
										**lk_params)

	# Select good points
	good_new = p1[st == 1]
	good_old = p0[st == 1]

	# draw the tracks
	for i, (new, old) in enumerate(zip(good_new,
									good_old)):
		a, b = new.ravel()
		c, d = old.ravel()
		mask = cv2.line(mask, (a, b), (c, d),
						color[i].tolist(), 2)
		
		frame = cv2.circle(frame, (a, b), 5,
						color[i].tolist(), -1)
		
	img = cv2.add(frame, mask)

	cv2.imshow('frame', img)
	
	k = cv2.waitKey(25)
	if k == 27:
		break

	# Updating Previous frame and points
	old_gray = frame_gray.copy()
	p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type


In [ ]:
import os
# get flownet2-pytorch source
!git clone https://github.com/Gauravv97/flownet2-pytorch.git
!mv /content/flownet2-pytorch /content/flownet2pytorch
os.chdir('./flownet2pytorch')
# install custom layers
!bash install.sh